In [28]:
import numpy as np
import pandas as pd
import json
from geojson import Point, Feature, FeatureCollection, dump

In [55]:
df = pd.read_csv('../raw/mpv-data.csv')
#df = df[['date', 'city', 'state', 'race', 'zip', 'county']]
print(df.dtypes)
df['zip'] = df['zip'].astype('Int64')
df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].astype('string')
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
print(df['longitude'].isnull().sum())
print(df['latitude'].isnull().sum())
empty_list = list(df['latitude'].isnull())

for index, x in enumerate(empty_list):
    if x == True:
        df.drop(index, inplace = True)
        
print(df['longitude'].isnull().sum())
print(df['latitude'].isnull().sum())
#df = df.fillna(np.nan).replace([np.nan], [None])
df.dropna(axis = 1, how = 'any', inplace = True)
df_13 = df[(df['date'] > '2013-01-01') & (df['date'] < '2014-01-01')]
#df_13
df_14 = df[(df['date'] > '2014-01-01') & (df['date'] < '2015-01-01')]
df_13

name                          object
age                          float64
gender                        object
race                          object
victim_image                  object
                              ...   
prosecutor_term               object
prosecutor_in_court           object
prosecutor_special            object
independent_investigation     object
prosecutor_url                object
Length: 62, dtype: object
1
1
0
0


,name,date,state,latitude,longitude
9367,Dontae Daveon Lewis Hayes,2013-12-31,CA,33.921347,-117.447669
9368,Ricky Junior Toney,2013-12-31,NC,36.261727,-78.281665
9369,James M. Eshelman,2013-12-30,WA,47.107991,-119.278546
9370,Christopher George,2013-12-29,CO,40.125028,-105.133476
9371,Jaspal Singh,2013-12-29,CA,35.349663,-118.946829
...,...,...,...,...,...
10443,Darrell Banks,2013-01-04,PA,39.990364,-75.162614
10444,Peter Jourdan,2013-01-03,NY,40.633157,-73.996953
10445,Chandler T. Guinn Jr.,2013-01-03,MO,39.032711,-94.575779
10446,Hunter Jacob Todd,2013-01-03,CA,38.711112,-121.263785


In [52]:
features = df_13.apply(
    lambda row: Feature(geometry=Point((float(row['longitude']), float(row['latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = df_13.drop(['latitude', 'longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('../final/mpv-shape13.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [54]:
features = df_14.apply(
    lambda row: Feature(geometry=Point((float(row['longitude']), float(row['latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = df_14.drop(['latitude', 'longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('../final/mpv-shape14.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [56]:
def df_to_geojson(df_13, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df_13.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [60]:
df_13_col = list(df_13.columns)
geojson_13 = df_to_geojson(df_13, df_13_col)

with open('mpv_13.json', 'w') as fp:
    json.dump(geojson_13, fp)
    
geojson_13

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Dontae Daveon Lewis Hayes',
    'date': '2013-12-31',
    'state': 'CA',
    'latitude': 33.921347,
    'longitude': -117.447669},
   'geometry': {'type': 'Point', 'coordinates': [-117.447669, 33.921347]}},
  {'type': 'Feature',
   'properties': {'name': 'Ricky Junior Toney',
    'date': '2013-12-31',
    'state': 'NC',
    'latitude': 36.261727,
    'longitude': -78.281665},
   'geometry': {'type': 'Point', 'coordinates': [-78.281665, 36.261727]}},
  {'type': 'Feature',
   'properties': {'name': 'James M. Eshelman',
    'date': '2013-12-30',
    'state': 'WA',
    'latitude': 47.107991,
    'longitude': -119.278546},
   'geometry': {'type': 'Point', 'coordinates': [-119.278546, 47.107991]}},
  {'type': 'Feature',
   'properties': {'name': 'Christopher George',
    'date': '2013-12-29',
    'state': 'CO',
    'latitude': 40.125028,
    'longitude': -105.133476},
   'geometry': {'type': 'Point', '

In [18]:
out_file = open("../clean/mpv_shape.geojson", "w") 
    
geojson.dump(geojson, out_file, indent = 6) 
    
out_file.close() 

TypeError: Object of type module is not JSON serializable

In [22]:
fips = pd.read_csv('US_FIPS_Codes.csv')
fips.rename(columns = {'FIPS State': 'state-fips', 'FIPS County': 'county-fips',
                      'County Name': 'county'}, inplace = True)
fips['state-fips'] = fips['state-fips'].apply(str)
fips['state-fips'] = fips['state-fips'].str.zfill(2)
fips['county-fips'] = fips['county-fips'].apply(str)
fips['county-fips'] = fips['county-fips'].str.zfill(3)
fips['fips'] = fips['state-fips'] + fips['county-fips']
print(fips.dtypes)
fips

State          object
county         object
state-fips     object
county-fips    object
fips           object
dtype: object


,State,county,state-fips,county-fips,fips
0,Alabama,Autauga,01,001,01001
1,Alabama,Baldwin,01,003,01003
2,Alabama,Barbour,01,005,01005
3,Alabama,Bibb,01,007,01007
4,Alabama,Blount,01,009,01009
...,...,...,...,...,...
3137,Wyoming,Sweetwater,56,037,56037
3138,Wyoming,Teton,56,039,56039
3139,Wyoming,Uinta,56,041,56041
3140,Wyoming,Washakie,56,043,56043


In [40]:
state_abbrev = pd.read_csv('state-abbrev.csv')
state_abbrev.rename(columns = {'Code':'state'}, inplace = True)
state_abbrev
fips_merge = pd.merge(state_abbrev, fips, 
                      on = 'State', how = 'outer')
fips_merge

,State,Abbrev,state,county,state-fips,county-fips,fips
0,Alabama,Ala.,AL,Autauga,01,001,01001
1,Alabama,Ala.,AL,Baldwin,01,003,01003
2,Alabama,Ala.,AL,Barbour,01,005,01005
3,Alabama,Ala.,AL,Bibb,01,007,01007
4,Alabama,Ala.,AL,Blount,01,009,01009
...,...,...,...,...,...,...,...
3137,Wyoming,Wyo.,WY,Sweetwater,56,037,56037
3138,Wyoming,Wyo.,WY,Teton,56,039,56039
3139,Wyoming,Wyo.,WY,Uinta,56,041,56041
3140,Wyoming,Wyo.,WY,Washakie,56,043,56043


In [41]:
df_tot = pd.merge(df, fips_merge, on = ['county', 'state'], how = 'outer')
df_tot

,date,city,state,race,zip,county,State,Abbrev,state-fips,county-fips,fips
0,2022-06-23,Jackson Township,PA,White,18321,Monroe,Pennsylvania,Pa.,42,089,42089
1,2021-10-19,Pocono Township,PA,White,18355,Monroe,Pennsylvania,Pa.,42,089,42089
2,2020-12-30,Stroudsburg,PA,Native Hawaiian/Other Pacific Islander,18360,Monroe,Pennsylvania,Pa.,42,089,42089
3,2020-07-06,Smithfield Township,PA,Unknown race,18301,Monroe,Pennsylvania,Pa.,42,089,42089
4,2020-03-07,Coolbaugh Township,PA,Unknown race,18466,Monroe,Pennsylvania,Pa.,42,089,42089
...,...,...,...,...,...,...,...,...,...,...,...
11882,NaT,NaN,WY,NaN,<NA>,Sublette,Wyoming,Wyo.,56,035,56035
11883,NaT,NaN,WY,NaN,<NA>,Sweetwater,Wyoming,Wyo.,56,037,56037
11884,NaT,NaN,WY,NaN,<NA>,Teton,Wyoming,Wyo.,56,039,56039
11885,NaT,NaN,WY,NaN,<NA>,Washakie,Wyoming,Wyo.,56,043,56043
